In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json, math, time
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb
import os

In [2]:
with open('/Users/ashley/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [3]:
tmdb.API_KEY = login['api-key']

In [4]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'tmdb_api_results_2001.json',
 'final_tmdb_data_2002.csv.gz',
 'title_basics.csv.gz',
 'tmdb_api_results_2002.json',
 'tmdb_api_results_2003.json',
 'final_tmdb_data_2001.csv.gz',
 'tmdb_api_results_2004.json',
 'title_akas.csv.gz',
 'tmdb_results_combined.csv.gz',
 'final_tmdb_data_2003.csv.gz',
 'title_ratings.csv.gz']

In [5]:
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0034413,False,Youth Gets a Break,Youth Gets a Break,0,2001,NaN,20,Short
1,tt0035423,False,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
2,tt0040241,False,Color Rhapsodie,Color Rhapsodie,0,2021,NaN,6,Short
3,tt0044326,False,Abstronic,Abstronic,0,2021,NaN,6,Short
4,tt0044879,False,Mandala,Mandala,0,2021,NaN,3,Short


In [6]:
YEARS_TO_GET =[2000,2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
               2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 
               2020, 2021, 2022]

#The rest of the years keep stopping I will do these in parts on the future

In [7]:
def write_json(new_data, filename):
    with open(filename, 'r+') as file:
        #load existing data
        file_data = json.load(file)
        #choose extend or append
        if (type(new_data)==list) & (type(file_data)==list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        #set files current position at offset
        file.seek(0)
        #convert back to json
        json.dump(file_data, file)

In [8]:
def get_movie_with_rating(movie_id):
    #get the movie and release dates
    movie = tmdb.Movies(movie_id)
    ##contruct output dict
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            movie_info['certification'] = c['certification']
    return movie_info

In [ ]:
#set up outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS',
                          position=0):
    #define the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    #check if file exists
    file_exists = os.path.isfile(JSON_FILE)

    #if it does not exist: create it
    if file_exists==False:
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}], f)

    #save new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    #save movie ids to list
    movie_ids = df['tconst'].copy()
    movie_ids

    #load existing data from json into dataframe
    previous_df = pd.read_json(JSON_FILE)

    #filter out any IDS in JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    for movie_id in tqdm_notebook(movie_ids_to_get, desc=f'Movies from {YEAR}', 
                                 position = 1, leave=True):
        try:
            temp = get_movie_with_rating(movie_id)
            #append/extend results to existing file
            write_json(temp, JSON_FILE)
            time.sleep(0.02)

            #if it fails
        except Exception as e:
            continue
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f'{FOLDER}final_tmdb_data_{YEAR}.csv.gz',compression='gzip', index=False)

YEARS:   0%|          | 0/19 [00:00<?, ?it/s]

Movies from 2004:   0%|          | 0/10339 [00:00<?, ?it/s]

Movies from 2005:   0%|          | 0/13414 [00:00<?, ?it/s]